**Báo cáo cuối kỳ môn học: PYTHON CHO KHOA HỌC DỮ LIỆU**

**Lớp 23TTH, Khoa Toán - Tin học, Trường Đại học Khoa học Tự nhiên, ĐHQG-HCM**

**Đề tài thực hiện:**
$$
\text{\textbf{USING DEEP LEARNING TO CLASSIFY ANIMAL AND HUMAN IMAGES}}
$$

**Giảng viên hướng dẫn: ThS. Hà Văn Thảo**

**Danh sách thành viên nhóm:**

1. 23110114 - Nguyễn Thị Hồng Thắm \
2. 23110123 - Lê Huỳnh Yến Vy \
3. 23110132 - Trần Nhật Anh

## GIỚI THIỆU

Object detection là một trong những chủ đề "nóng" trong deep learing bởi tính ứng dụng cao trong thực tiễn và nguồn dữ liệu dồi dào, dễ chuẩn bị. Một trong những thuật toán object detection nổi tiếng nhất là **YOLO**.

YOLO là mô hình mạng neuron tích chập (CNN) được sử dụng phổ biển để nhận dạng các đối tượng trong ảnh hoặc video. Điểm đặc biệt của mô hình này là có khả năng phát hiện tất cả các đối tượng trong một hình ảnh chỉ qua một lần lan truyền của CNN.

Các phương pháp truyền thống tách biệt bước đề xuất vùng và bước phân loại, YOLO xử lý đầu vào, vừa phân loại được các đối tượng, vừa dự đoán được vị trí của chúng trong một lần duy nhất.

YOLO có nghĩa là "You only look once", nghĩa là chỉ cần "nhìn" một lần là thuật toán đã có thể phát hiện được vật thể, cho thấy độ nhanh của thuật toán gần như là real-time.

Ứng dụng của YOLO cũng như nhiều thuật toán object detection khác, rất đa dạng: quản lý giao thông, đếm số sản phẩm trên băng chuyền nhà máy, đếm số vật nuôi trong chăn nuôi, phát hiện vật thể nguy hiểm (súng, dao,...), chấm công tự động,...

## TẠO MÔI TRƯỜNG ẢO VÀ KERNEL CHẠY NOTEBOOK (LINUX)

Dự án Python cần **môi trường ảo (virtual environment)** để tự cách ly, tránh xung đột phiên bản thư viện giữa các dự án. `venv` là môi trường ảo mà chúng ta sẽ sử dụng trong dự án này. Sau khi cài đặt `venv`, chúng ta di chuyển đường dẫn đến folder chứa dự án trong terminal và sử dụng lệnh sau để cài đặt môi trường ảo cho dự án:

`python -m venv .venv`

Trong đó, `.venv` là tên của folder chứa môi trường ảo của dự án, đồng thời nó cũng sẽ "đóng băng" phiên bản Python, pip và các thư viện sẽ được dùng trong dự án.

Kích hoạt môi trường ảo:

`source .venv/bin/activate`

Lúc này, phiên bản Python và `pip` được dùng là của môi trường ảo, các thư viện cài bằng `pip install` cũng chỉ ảnh hưởng trong `.venv`. Cách nhận biết đang ở môi trường ảo là promt terminal thường đổi thành `(.venv) user_name@machine:~` (nếu đang sử dụng Linux). Khi đã kích hoạt môi trường ảo, đảm bảo phiên bản Python và `pip` đã "đóng băng" trong đó, sử dụng lệnh:

`which python && which pip`

Nếu output có dạng `.../<project_name>/.venv/...` thì môi trường ảo đã được kích hoạt thành công.

Tiếp theo, tạo một kernel để chạy Jupyter Notebook. Cài đặt `ipykernel` để tạo kernel:

`python -m pip install ipykernel`

Sau khi cài đặt thành công, tiến hành tạo kernel để chạy file `.ipynb`:

`python -m ipykernel install --prefix .venv --name yolovenv --display-name "this_project"`

`--prefix .venv`: kernel mặc định không tự lưu vào `.venv`, thuộc tính này sẽ lưu kernel đã tạo vào `.venv`  
`--name yolovenv`: tên folder chứa kernel, ở đây tên folder là `yolovenv`. Kernel sẽ được lưu tại `.venv/share/jupyter/kernels/yolovenv/`  
`--display-name "this_project`: kernel sẽ hiển thị dưới tên `this_project` trong VS Code.

Khi đã tạo kernel, click vào biểu tượng kernel ở góc trên bên phải, chọn
$$
\text{Select Another Kernel} \rightarrow \text{Jupyter Kernel...} \rightarrow \text{this\_project}
$$

## KHAI BÁO THƯ VIỆN VÀ CHUẨN BỊ DỮ LIỆU

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ultralytics import YOLO

from pathlib import Path

## KHAI BÁO, HUẤN LUYỆN VÀ LƯU MÔ HÌNH

Hàm train model.

In [2]:
def train_model(model, dataset_path, project_path, project_name):
    model.train(
        data=dataset_path,
        epochs=30,
        imgsz=416,  # Nếu train -> 640
        batch=4,  # Nếu ổn -> 8
        workers=2, # Nếu ổn -> 4
        device=0,  # Sử dụng GPU
        project=project_path,
        name=project_name
    )

Sử dụng model YOLO11s.

In [3]:
BASE_MODEL = "yolo11s.pt"

Huấn luyện.

In [7]:
DATASET_PATH = "dataset.yaml"

# Lưu model tại "runs/yolo11s_custom/my_model" sau khi train
MODEL_PATH = Path("runs/yolo11s_custom/my_model/weights/best.pt")
PROJECT_PATH = "runs/yolo11s_custom"
PROJECT_NAME = "my_model"

# Huấn luyện mô hình với dataset
if MODEL_PATH.exists():
    print(f"Model has been trained already. It is being loaded again: {MODEL_PATH}")
    model = YOLO(str(MODEL_PATH))
else:
    print("Model hasn't been trained. Start training...")

    # Train model dựa trên model gốc là YOLO11s
    model = YOLO(BASE_MODEL)
    train_model(model, DATASET_PATH, PROJECT_PATH, PROJECT_NAME)

    # Load lại best.pt sau khi train, nếu không tìm thấy thì in ra lỗi
    assert MODEL_PATH.exists(), "Training finished but file best.pt not found"
    model = YOLO(str(MODEL_PATH))

    print("Training finished")

# Kiểm tra và khoá model với model gốc là YOLO11s
# assert model.model.yaml['name'] == 'yolo11s'
# model.info()

Model has been trained already. It is being loaded again: runs/yolo11s_custom/my_model/weights/best.pt


In [9]:
data = pd.read_csv("runs/yolo11s_custom/my_model/results.csv")
data.head(5)
data.tail(5)

,epoch,time,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
25,26,16514.7,0.58776,0.50383,0.98996,0.54904,0.26710,0.73119,0.63142,0.67298,0.78483,0.98931,0.000021,0.000021,0.000021
26,27,17146.4,0.57937,0.47806,0.98274,0.53217,0.25831,0.72728,0.62576,0.67011,0.78188,0.98571,0.000017,0.000017,0.000017
27,28,17777.9,0.56971,0.45596,0.97951,0.53232,0.25831,0.72484,0.62557,0.66932,0.79451,0.98486,0.000013,0.000013,0.000013
28,29,18409.4,0.56665,0.44172,0.97295,0.54233,0.26273,0.72901,0.62933,0.66458,0.78846,0.98169,0.000009,0.000009,0.000009
29,30,19041.6,0.55800,0.42735,0.97055,0.54828,0.26573,0.72750,0.62879,0.66602,0.79150,0.98473,0.000005,0.000005,0.000005


## TEST